# **Доклад по библиотеке SymPy**

![SymPy Report](https://raw.githubusercontent.com/vikvlr/SymPy_report/main/picture.jpg)

**SymPy** — это библиотека для символьных вычислений на Python. Её ключевая особенность: работа не с числами (как в NumPy), а с математическими выражениями как полноценными объектами. Мы взяли эту тему, так как неоднократно встречались с этой темой в наших лабораторных работах по матанализу и хотим рассказать о ней поподробнее

### Ключевые возможности:
- **Символьные вычисления**: работа с переменными как с математическими символами
- **Алгебраические преобразования**: упрощение, раскрытие скобок, факторизация
- **Математический анализ**: пределы, производные, интегралы, ряды
- **Решение уравнений**: алгебраические, дифференциальные, системы уравнений
- **Линейная алгебра**: матричные операции, собственные значения, определители
- **Дискретная математика**: комбинаторика, теория графов, логические выражения
- **Визуализация**: построение графиков, вывод в LaTeX


### Основные компоненты API:
1. **Символы (Symbols)** - базовые строительные блоки
2. **Выражения (Expressions)** - комбинации символов и операций
3. **Функции (Functions)** - математические функции (sin, cos, exp, log)
4. **Матрицы (Matrices)** - линейно-алгебраические структуры
5. **Решатели (Solvers)** - для уравнений и систем
6. **Утилиты** - упрощение, подстановка, численная оценка

## Документация

Документация по библиотеке SymPy находится по данной ссылочке: https://docs.sympy.org/latest/index.html

## Установка библиотеки

Официально рекомендуемым методом установки пакетов Python из PyPi является через pip:

In [ ]:
%pip install sympy

## Техническая реализация библиотеки

### Древовидная структура

Представьте, у нас есть выражение (x + 3) * (y - 2). Внутри SymPy хранит его вот так:

          Умножение (*)
           /         \
       Сложение (+)  Вычитание (-)
         /  \         /  \
        x    3       y    2

Корень дерева - операция умножения. У него два аргумента: (x + 3) и (y - 2). Каждый из этих аргументов - тоже дерево. Давайте посмотрим, как это выглядит в коде:

In [ ]:
from sympy import symbols, srepr

x, y = symbols('x y')

expr = (x + 3) * (y - 2)

print("Внутреннее представление выражения:")
print(srepr(expr))

Внутреннее представление выражения:
Mul(Add(Symbol('x'), Integer(3)), Add(Symbol('y'), Integer(-2)))


Чтобы лучше визуализировать дерево, напишем простую функцию:

In [ ]:
def show_tree(expr, level=0):
    indent = "  " * level
    node_name = type(expr).__name__
    print(f"{indent}└── {node_name}: {expr}")

    for arg in expr.args:
        show_tree(arg, level + 1)

print("\nДерево выражения (x + 3) * (y - 2):")
show_tree(expr)


Дерево выражения (x + 3) * (y - 2):
└── Mul: (x + 3)*(y - 2)
  └── Add: y - 2
    └── Integer: -2
    └── Symbol: y
  └── Add: x + 3
    └── Integer: 3
    └── Symbol: x


Эта визуализация помогает понять структуру:

- На верхнем уровне операция умножения (Mul)

- Первый аргумент умножения: сложение (x + 3)

- Второй аргумент умножения: вычитание (y - 2), представленное как сложение


Еще один более сложный пример, чтобы лучше понять:

In [ ]:
complex_expr = x**2 + 2*x*y + y**2
print("\nСложное выражение: x² + 2xy + y²")
print("Его дерево:")
show_tree(complex_expr)


Сложное выражение: x² + 2xy + y²
Его дерево:
└── Add: x**2 + 2*x*y + y**2
  └── Pow: x**2
    └── Symbol: x
    └── Integer: 2
  └── Pow: y**2
    └── Symbol: y
    └── Integer: 2
  └── Mul: 2*x*y
    └── Integer: 2
    └── Symbol: x
    └── Symbol: y


### Иерархия классов

Все в SymPy наследуется от базового класса **Basic**. Это фундаментальный принцип библиотеки, который обеспечивает единое поведение для всех математических объектов.

Вот как выглядит упрощенная иерархия классов:

     Basic
     ├── Atom (неделимые объекты)
     │    ├── Symbol (переменные: x, y)
     │    └── Number (числа: 1, 2.5, π)
     └── Expr (выражения)
          ├── Add (сложение: a + b)
          ├── Mul (умножение: a * b)
          ├── Pow (степень: a**b)
          └── Function (функции: sin, cos)

  

### Иммутабельность

Одна из ключевых особенностей SymPy — все объекты неизменяемы (иммутабельны). Это значит, что после создания объекта его нельзя изменить.

In [ ]:
from sympy import Symbol

x = Symbol('x')
expr = x + 5

# Попробуем изменить
try:
    expr.args = (x, x)  # Ошибка!
except AttributeError:
    print("Объекты SymPy нельзя изменять после создания")

# Вместо этого создаем новый объект
new_expr = x + x

Объекты SymPy нельзя изменять после создания


### Механизм хэширования

Хэширование — это процесс преобразования объекта в число (хэш). В SymPy это нужно, чтобы выражения можно было использовать как ключи в словарях или хранить в множествах.

In [ ]:
# Два одинаковых выражения имеют одинаковый хэш
expr1 = x + 1
expr2 = 1 + x

print(f"expr1 == expr2: {expr1 == expr2}")
print(f"hash(expr1) == hash(expr2): {hash(expr1) == hash(expr2)}")

# Разные выражения - разные хэши
expr3 = x + 2
print(f"hash(expr1) == hash(expr3): {hash(expr1) == hash(expr3)}")

expr1 == expr2: True
hash(expr1) == hash(expr2): True
hash(expr1) == hash(expr3): False


Как работает хэширование (упрощенная реализация из SymPy):

In [ ]:
class Basic:
    def __hash__(self):
        # Хэш = хэш от (тип объекта, его аргументы)
        return hash((type(self), self.args))

Это означает, что два выражения считаются одинаковыми (и имеют одинаковый хэш), если у них одинаковый тип и одинаковые аргументы.

### Кэширование (оптимизация)

SymPy использует кэширование для оптимизации работы. Когда вы создаете выражение, SymPy проверяет, не создавали ли вы такое же выражение раньше.

In [ ]:
from sympy import Add

# Эти два выражения - один и тот же объект в памяти
a = Add(x, 1)
b = Add(x, 1)

print(f"a is b (один объект?): {a is b}")
print(f"ID объекта a: {id(a)}")
print(f"ID объекта b: {id(b)}")

a is b (один объект?): True
ID объекта a: 136720509628800
ID объекта b: 136720509628800


Преимущества кэширования:

- Экономия памяти — не нужно хранить одинаковые объекты несколько раз
- Быстрое сравнение — если объекты физически одинаковы (тот же адрес в памяти), то они точно равны
- Предотвращение дублирования — гарантия, что одинаковые выражения представлены одним объектом

## Использование библиотеки



#### Символы


Символы представляют из себя математические переменные, с которыми мы можем работать символически.

In [ ]:
from sympy import symbols

x, y, z = symbols('x y z')
a, b, c = symbols('a b c')

#### Базовые операции с выражениями

SymPy позволяет создавать, упрощать и преобразовывать математические выражения различными способами.

In [ ]:
from sympy import simplify, expand, factor, together

Создание выражений

In [ ]:
expr1 = x**2 + 2*x + x
expr1

x**2 + 3*x

Упрощение выражений

In [ ]:
simplify(expr1)

x*(x + 3)

Раскрытие скобок

In [ ]:
expand((x + 1)**3)

x**3 + 3*x**2 + 3*x + 1

Факторизация

In [ ]:
factor(x**2 - 4)

(x - 2)*(x + 2)

Объединение дробей

In [ ]:
together(1/x + 1/y)

(x + y)/(x*y)

#### Математический анализ

Библиотека предоставляет полный набор инструментов для дифференцирования, интегрирования и работы с пределами.

In [ ]:
from sympy import diff, integrate, limit, series, sin, cos, exp, log

Производные

In [ ]:
diff(sin(x)*exp(x), x)

exp(x)*sin(x) + exp(x)*cos(x)

Частные производные

In [ ]:
diff(x**2*y + y**2, x, y)

2*x

Интегралы

In [ ]:
integrate(cos(x), x)
integrate(x**2, (x, 0, 1))

1/3

Пределы

In [ ]:
limit(sin(x)/x, x, 0)

1

Разложение в ряд Тейлора

In [ ]:
series(exp(x), x, 0, 5)

1 + x + x**2/2 + x**3/6 + x**4/24 + O(x**5)

#### Решение уравнений и систем

SymPy умеет решать алгебраические, тригонометрические и дифференциальные уравнения.

In [ ]:
from sympy import solve, nsolve, linsolve, Eq

Алгебраические уравнения

In [ ]:
solve(Eq(x**2 - 4, 0), x)

[-2, 2]

Тригонометрические уравнения

In [ ]:
solve(Eq(sin(x), 0), x)

[0, pi]

Системы линейных уравнений

In [ ]:
linsolve([Eq(x + y, 3), Eq(2*x - y, 0)], [x, y])

{(1, 2)}

#### Матрицы и линейная алгебра

SymPy поддерживает все основные матричные операции и методы линейной алгебры.

In [ ]:
from sympy import Matrix, zeros, ones, diag

Создание матриц

In [ ]:
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[2, 0], [1, 2]])

Базовые операции

In [ ]:
# Сложение
A + B

Matrix([
[3, 2],
[4, 6]])

In [ ]:
# Умножение
A * B

Matrix([
[ 4, 4],
[10, 8]])

Специальные матрицы

In [ ]:
zeros(2, 3)

Matrix([
[0, 0, 0],
[0, 0, 0]])

In [ ]:
# Единичная матрица
ones(2, 2)

Matrix([
[1, 1],
[1, 1]])

Детерминант

In [ ]:
A.det()

-2

Обратная матрица

In [ ]:
A.inv()

Matrix([
[ -2,    1],
[3/2, -1/2]])

#### Дискретная математика

Библиотека включает инструменты для комбинаторики (дискра) и теории чисел.

In [ ]:
from sympy import factorial, binomial, fibonacci, gcd, lcm

Факториал и биномиальные коэффициенты

In [ ]:
factorial(10)

3628800

In [ ]:
binomial(10, 3)

120

Числа Фибоначчи

In [ ]:
fibonacci(10)

55

Наибольший общий делитель и наименьшее общее кратное

In [ ]:
gcd(48, 180)

12

In [ ]:
lcm(48, 180)

720

#### Геометрия

SymPy позволяет работать с геометрическими объектами.

In [ ]:
from sympy import Point, Line, Circle, Triangle, Segment

Точки

In [ ]:
p1 = Point(0, 0)
p2 = Point(1, 1)
p3 = Point(2, 0)

Прямые и отрезки

In [ ]:
line = Line(p1, p2)
segment = Segment(p1, p2)

Окружности и треугольники

In [ ]:
circle = Circle(p1, 2)
triangle = Triangle(p1, p2, p3)

#### Теория вероятностей и статистика

SymPy предоставляет инструменты для работы с вероятностными распределениями и статистикой.

In [ ]:
from sympy.stats import Normal, Uniform, Exponential, E, variance, density

Нормальное распределение

In [ ]:
X = Normal('X', 0, 1)

In [ ]:
# Математическое ожидание
E(X)

0

In [ ]:
# Дисперсия
variance(X)

1

Равномерное распределение

In [ ]:
Y = Uniform('Y', 0, 1)

Экспоненциальное распределение

In [ ]:
Z = Exponential('Z', 1)

Мы рассмотрели лишь часть основных функций библиотеки, и на самом деле в библиотеке функций намного больше, и они охватывают практически все области математики. Библиотека SymPy очень полезна в учебном процессе, так как позволяет выполнять сложные символьные вычисления, которые мы очень часто используем в лабораторных работах по математическому анализу и линейной алгебре.